In [ ]:
import sys

sys.path.append("..")

In [ ]:
train = pd.read_csv("../data/processed/train.csv", sep=",")
# test = pd.read_csv("../data/raw/test.csv", sep=",")

print("The shape of train is {}".format(train.shape))
# print("The shape of test is {}".format(test.shape))

In [ ]:
from src.transformers import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
ct = ColumnTransformer(
    [
        (
            "ohe",
            OneHotEncoder(sparse=False, handle_unknown="ignore"),
            ["type_of_vehicle", "transmission", "make_model"],
        ),
    ],
    remainder="passthrough",
)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

pipe = Pipeline(
    [
        ("add_cat", SplitValuesToColumn("category")),
        ("post_processing", PostProcessing()),
        ("normalization", ct),
    ]
)

In [ ]:
trainX = train.loc[:, train.columns != "price"]
trainY = train.loc[:, "price"]

X_train = pipe.fit_transform(trainX)
# X_test = pipe.transform(test)
y_train = trainY

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    trainX, trainY, test_size=0.2, random_state=72
)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)

y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [ ]:
rfr_hpo = {
    "max_depth": [40, 50, 60, 100],
    "max_features": ["sqrt"],
    "n_estimators": [100, 200, 400],
}

gb_hpo = {
    "n_estimators": [200, 400],
    "learning_rate": [0.01, 0.1],
    "max_depth": [40, 60, 100],
    "max_features": ["sqrt"],
}
xgb_hpo = {"n_estimators": [200, 400, 600], "max_depth": [40, 60, 100]}


space = dict()
space["solver"] = ["svd", "cholesky", "lsqr", "sag"]
space["alpha"] = loguniform(1e-5, 100)
space["fit_intercept"] = [True, False]
space["normalize"] = [True, False]
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    BaggingRegressor,
)
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
# import xgboost as xgb

model = RandomForestRegressor
# model = GradientBoostingRegressor()
# model = Ridge()
# model = xgb.XGBRegressor
rf_random = RandomizedSearchCV(
    estimator=model(),
    param_distributions=rfr_hpo,
    n_iter=20,
    cv=3,
    verbose=3,
    random_state=42,
    n_jobs=-1,
)

In [ ]:
rf_random.fit(X_train, y_train)
best_model = RandomForestRegressor(**rf_random.best_params_)
# best_model = xgb.XGBRegressor(n_estimators=200,
#                              eta=0.2, gamma=1, max_depth=50,
#                              reg_lambda=1, n_jobs=-1)
best_model.fit(X_train, y_train)

In [ ]:
y_pred = best_model.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)